## Introduction:
Work with data from the S&P500 Index. 




### Data Dictionary:
|Feature|Description|
|:----|:----|
|Date | The date of the record.|
|Open | The opening price of the day (when trading starts).|
|High | The highest trade price during the day.|
|Low | The lowest trade price during the day.|
|Close | The closing price for the day (when trading is finished).|
|Volume | The number of shares traded.|
|Adj Close | The daily closing price, adjusted retroactively to include any corporate actions.|

## Goals:
Use historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index goes up or down helps forecast how the stock market as a whole performs. Since stocks tend to correlate with how well the economy as a whole is performs, it can also help with economic forecasts.

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime

In [48]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score

### Adjust View Settings

In [3]:
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 100)

## Loading Dataset

In [4]:
stock = pd.read_csv('sphist.csv')

In [5]:
stock.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [6]:
stock.tail()

,Date,Open,High,Low,Close,Volume,Adj Close
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66


In [7]:
stock.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], dtype='object')

### Summary Statistics and Exploration

In [8]:
stock.shape

(16590, 7)

In [9]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  float64
 6   Adj Close  16590 non-null  float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


In [10]:
stock.describe()

,Open,High,Low,Close,Volume,Adj Close
count,16590.000000,16590.000000,16590.000000,16590.000000,1.659000e+04,16590.000000
mean,482.570941,485.624237,479.367501,482.692491,7.940099e+08,482.692491
std,554.889186,558.186049,551.367625,555.007904,1.456582e+09,555.007904
min,16.660000,16.660000,16.660000,16.660000,6.800000e+05,16.660000
25%,83.860001,84.594997,83.139997,83.860001,7.610000e+06,83.860001
50%,144.049995,145.294998,143.105004,144.264999,7.170500e+07,144.264999
75%,950.722488,956.665024,941.969986,950.797500,7.866750e+08,950.797500
max,2130.360107,2134.719971,2126.060059,2130.820068,1.145623e+10,2130.820068


In [11]:
stock.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
dtype: int64

Lets convert the column *Date* to datetime object

In [12]:
stock['Date'] = pd.to_datetime(stock['Date'])

In [13]:
# Sort the dataframe in ascending order by date
stock = stock.sort_values(by='Date')
stock.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


## Generating Indicators

### A note on generating Indicators:

Datasets taken from the stock market need to be handled differently than datasets from other sectors when it's time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential and each observation comes a day after the previous observation. Thus, the observations are not all independent and we can't treat them as such.

This means we have to be extra careful not to inject "future" knowledge into past rows when you train and predict. Injecting future knowledge makes our model look good when we train and test it, but it fails in the real world. This is how many algorithmic traders lose money.

The time series nature of the data means that we can generate indicators to make our model more accurate. For instance, we can create a new column that contains the average price of the last 10 trades for each row. This incorporates information from multiple prior rows into one and makes predictions much more accurate.

When we do this, you have to be careful not to use the current row in the values you average. We want to teach the model how to predict the current price from historical prices. If we include the current price in the prices you average, it will be equivalent to handing the answers to the model upfront, and will make it impossible to use in the "real world", where we don't know the price upfront.

Here are some indicators that are interesting to generate for each row:

- The average price from the past 5 days.
- The average price for the past 30 days.
- The average price for the past 365 days.
- The ratio between the average price for the past 5 days, and the average price for the past 365 days.
- The standard deviation of the price over the past 5 days.
- The standard deviation of the price over the past 365 days.
- The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.

"Days" means "trading days" -- so if we are computing the average of the past 5 days, it should be the 5 most recent dates before the current one. Assume that "price" means the Close column. We have to be careful not to include the current price in these indicators! We are predicting the next day price, so our indicators are designed to predict the current price from the previous prices.

In [14]:
stock.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


### Indicators for Average Prices, Standard Deviation of Prices and Ratios 

In [15]:
# Prices mean for previous 5, 30, and 365 days 
stock['5_days_mean'] = stock['Close'].rolling(5).mean().shift(1)
stock['30_days_mean'] = stock['Close'].rolling(30).mean().shift(1)
stock['365_days_mean'] = stock['Close'].rolling(365).mean().shift(1)

# Ratio 
stock['ratio_bw_5_365_mean'] = stock['5_days_mean'] / stock['365_days_mean']

In [16]:
# Prices Standard Deviation for previous 5, and 365 days
stock['5_days_std'] = stock['Close'].rolling(5).std().shift(1)
stock['365_days_std'] = stock['Close'].rolling(365).std().shift(1)

# Ratio
stock['ratio_bw_5_365_std'] = stock['5_days_std'] / stock['365_days_std']

In [17]:
stock.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5_days_mean,30_days_mean,365_days_mean,ratio_bw_5_365_mean,5_days_std,365_days_std,ratio_bw_5_365_std
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
stock.tail()

,Date,Open,High,Low,Close,Volume,Adj Close,5_days_mean,30_days_mean,365_days_mean,ratio_bw_5_365_mean,5_days_std,365_days_std,ratio_bw_5_365_std
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,2073.984998,2035.531178,1.025297,3.916109,64.370261,0.060837
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,2076.283993,2035.914082,1.026680,7.956808,64.352527,0.123644
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,2077.908659,2036.234356,1.025573,9.333599,64.277554,0.145208
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,2078.931331,2036.507343,1.021580,19.599946,64.121622,0.305668
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068,2080.771973,2080.237329,2036.869425,1.021554,19.806136,64.058862,0.309187


Since We are computing indicators that use historical data, there are some rows where there is not enough historical data to generate them. 

We will need to remove those rows before splitting the data. 

We can expect to have the maximum number of rows with at least one null value in 365_days_mean and 365_days_std.

In [19]:
stock['365_days_mean'].isnull().sum()

365

In [20]:
stock[stock['Date'] > datetime(year=1951, month=1, day=3)]['365_days_mean'].isnull().sum()

114

In [21]:
stock = stock[stock['Date'] > datetime(year=1951, month=1, day=3)]

We still have some null values in the dataset

In [22]:
stock = stock.dropna(axis=0)
stock.shape

(16225, 14)

In [23]:
stock.head(100)

,Date,Open,High,Low,Close,Volume,Adj Close,5_days_mean,30_days_mean,365_days_mean,ratio_bw_5_365_mean,5_days_std,365_days_std,ratio_bw_5_365_std
16224,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.800000,21.703333,19.447726,1.120954,0.256223,1.790253,0.143121
16223,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.900000,21.683000,19.462411,1.125246,0.213659,1.789307,0.119409
16222,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.972000,21.659667,19.476274,1.128142,0.092574,1.788613,0.051758
16221,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.960000,21.631000,19.489562,1.126757,0.115108,1.787659,0.064390
16220,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,21.862000,21.599000,19.502082,1.121008,0.204132,1.786038,0.114293
16219,1951-06-26,21.299999,21.299999,21.299999,21.299999,1260000.0,21.299999,21.710000,21.564333,19.513617,1.112556,0.292831,1.783946,0.164148
16218,1951-06-27,21.370001,21.370001,21.370001,21.370001,1360000.0,21.370001,21.566000,21.535000,19.525315,1.104515,0.278980,1.781613,0.156589
16217,1951-06-28,21.100000,21.100000,21.100000,21.100000,1940000.0,21.100000,21.458000,21.522000,19.537041,1.098324,0.208015,1.779624,0.116887
16216,1951-06-29,20.959999,20.959999,20.959999,20.959999,1730000.0,20.959999,21.322000,21.502333,19.548932,1.090699,0.162080,1.775513,0.091286
16215,1951-07-02,21.100000,21.100000,21.100000,21.100000,1350000.0,21.100000,21.204000,21.470667,19.560685,1.084011,0.169205,1.770595,0.095564


## Splitting the Dataset into Train and Test:
- **train** should contain any rows in the data with a date **less than 2013-01-01** 
- **test** should contain any rows with a date **greater than or equal to 2013-01-01**

In [24]:
train = stock[stock['Date'] < datetime(year=2013, month=1, day=1)]

In [25]:
test = stock[stock['Date'] >= datetime(year=2013, month=1, day=1)]

### Error Metric
We will use Mean Absolute Error, also called MAE, as an error metric, because it will show how "close" we were to the price in intuitive terms. Mean Squared Error, or MSE, is an alternative that is more commonly used, but makes it harder to intuitively tell how far off we are from the true price because it squares the error.

### Features:
We will omit all original columns we had with the dataset, that includes *Close* as our **target variable**, as well as  *High, Low, Open, Volume, Adj Close, Date* when training our model.
These all contain knowledge of the future that we don't want to feed the model.

In [86]:
features = stock.columns.tolist()
for col in ['High', 'Low', 'Open', 'Volume', 'Adj Close', 'Date', 'Close']:
    features.remove(col)

In [87]:
features

['5_days_mean',
 '30_days_mean',
 '365_days_mean',
 'ratio_bw_5_365_mean',
 '5_days_std',
 '365_days_std',
 'ratio_bw_5_365_std']

In [37]:
target = 'Close'

In [89]:
all_features = features
all_features.append('Close')

In [90]:
all_features

['5_days_mean',
 '30_days_mean',
 '365_days_mean',
 'ratio_bw_5_365_mean',
 '5_days_std',
 '365_days_std',
 'ratio_bw_5_365_std',
 'Close']

## Linear Regression Model

In [53]:
# Instantiate the model
linreg = LinearRegression()

# Fit the model
linreg.fit(train[features], train[target])

# Predict
pred_train = linreg.predict(train[features])
pred_test = linreg.predict(test[features])

# Error
mae_train = mean_absolute_error(train[target], pred_train)
mae_test = mean_absolute_error(test[target], pred_test)

mse_train = mean_squared_error(train[target], pred_train)
mse_test = mean_squared_error(test[target], pred_test)


print('MAE train: ',mae_train)
print('MAE test: ',mae_test)
print('-'*30)
print('MSE train: ',mse_train)
print('MSE test: ',mse_test)

MAE train:  4.979250385436781
MAE test:  16.145140609743574
------------------------------
MSE train:  108.6982660907699
MSE test:  492.9230344450398


In [40]:
stock.corr()['Close']

Open                   0.999900
High                   0.999953
Low                    0.999956
Close                  1.000000
Volume                 0.772817
Adj Close              1.000000
5_days_mean            0.999793
30_days_mean           0.999189
365_days_mean          0.988870
ratio_bw_5_365_mean    0.047782
5_days_std             0.722414
365_days_std           0.816103
ratio_bw_5_365_std     0.087018
Name: Close, dtype: float64

#### Lets consider taking the following features:
- Note that we already had omitted *Open, High, Low, Volume, Adj Close* and *Close* from our features for training
- Lets consider taking the following out as well, as they do not show significant correlation with the target variable
    - ratio_bw_5_365_mean
    - ratio_bw_5_365_mean

In [45]:
features_1 = []
for feature in features:
    if 'ratio' not in feature:
        features_1.append(feature)
print(features_1)

['5_days_mean', '30_days_mean', '365_days_mean', '5_days_std', '365_days_std']


In [52]:
# Instantiate the model
linreg = LinearRegression()

# Fit the model
linreg.fit(train[features_1], train[target])

# Predict
pred_train = linreg.predict(train[features_1])
pred_test = linreg.predict(test[features_1])

# Error
mae_train = mean_absolute_error(train[target], pred_train)
mae_test = mean_absolute_error(test[target], pred_test)

mse_train = mean_squared_error(train[target], pred_train)
mse_test = mean_squared_error(test[target], pred_test)


print('MAE train: ',mae_train)
print('MAE test: ',mae_test)
print('-'*30)
print('MSE train: ',mse_train)
print('MSE test: ',mse_test)

MAE train:  4.981706402946077
MAE test:  16.1460567745569
------------------------------
MSE train:  108.70663902551452
MSE test:  492.8297140338371


## Predicting only for the next day

In [148]:
def next_day(row):
    train = stock.iloc[:-row]
    test = stock.iloc[-row:]
    
    linreg = LinearRegression()
    linreg.fit(train[features], train[target])
    
    pred_train = linreg.predict(train[features])
    pred_test = linreg.predict(test[features])
    mse_train = mean_squared_error(train[target], pred_train)
    mse_test = mean_squared_error(test[target], pred_test)
    return mse_train, mse_test
    
    

In [149]:
next_day(10)

(7.14215337054402e-25, 5.169878828456423e-24)

# TBC - not completed yet